In [1]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split, GridSearchCV
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
order_items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/databases/day9/DNC_order_items_dataset.csv')

In [3]:
order_reviews = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/databases/day9/DNC_order_reviews_dataset.csv')

In [4]:
orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/databases/day9/DNC_orders_dataset.csv')

In [5]:
products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/databases/day9/DNC_products_dataset.csv')

In [132]:
order_items.head()

,Unnamed: 0,order_id,order_item_id,product_id,price
0,0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,R$58.90
1,1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,R$239.90
2,2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,R$199.00
3,3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,R$12.99
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,R$199.90


In [8]:
order_reviews.head()

,Unnamed: 0,review_id,order_id,review_score
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [9]:
orders.head()

,Unnamed: 0,order_id,customer_id,order_status
0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


In [28]:
orders['order_status'].unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [10]:
products.head()

,Unnamed: 0,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,225.0,16.0,10.0,14.0
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1000.0,30.0,18.0,20.0
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,154.0,18.0,9.0,15.0
3,3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,371.0,26.0,4.0,26.0
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,625.0,20.0,17.0,13.0


###Vamos fazer um algoritmo de recomendação de produtos baseado na categoria . Exemplo: estou olhando um tênis para comprar, e o site me recomenda um par de meias.

###O algoritmo me retornará a lista com os 10 produtos mais vendidos daquela categoria

#### Removemos as colunas não interessantes neste momento

In [41]:
product_clean = products.drop(['product_name_lenght','product_description_lenght','product_weight_g','product_length_cm','product_height_cm','product_width_cm','Unnamed: 0'], axis=1).reset_index()

In [42]:
product_clean.head()

,index,product_id,product_category_name
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,3,cef67bcfe19066a932b7673e239eb23d,bebes
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


### Removemos as categorias nan

In [43]:
product_clean.dropna(subset=['product_category_name'], inplace=True)

### Juntamos as tabelas de itens com a de itens comprados para obter os preços de venda dos itens

In [47]:
order_items_products = pd.merge(order_items, product_clean, how="inner", on="product_id")

In [53]:
order_items_products.head()

,Unnamed: 0,order_id,order_item_id,product_id,price,index,product_category_name
0,0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,25865,cool_stuff
1,8346,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,55.9,25865,cool_stuff
2,36646,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,64.9,25865,cool_stuff
3,49108,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,58.9,25865,cool_stuff
4,55021,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,58.9,25865,cool_stuff


### Aqui fizemos a contagem de vendas de cada produto agrupando por id do produto

In [78]:
top_product_by_category = pd.DataFrame(order_items_products['product_id'].value_counts().reset_index())

In [81]:
top_product_by_category.rename(columns={'index':'product_id', 'product_id':'quantity'}, inplace=True)

In [82]:
top_product_by_category.head()

,product_id,quantity
0,aca2eb7d00ea1a7b8ebd4e68314663af,527
1,99a4788cb24856965c36a24e339b6058,488
2,422879e10f46682990de24d770e7f83d,484
3,389d119b48cf3043d311335e499d9c6b,392
4,368c6c730842d78016ad823897a372db,388


### Criamos um novo dataframe juntando a lista de produtos com a quantidade de itens vendidos

In [85]:
df = product_clean.merge(top_product_by_category, how='left', on='product_id')

### Ordenamos para verificar se a contagem estava OK

In [89]:
df.sort_values(by=['quantity'], ascending=False)

,index,product_id,product_category_name,quantity
13173,13430,aca2eb7d00ea1a7b8ebd4e68314663af,moveis_decoracao,527
9480,9661,99a4788cb24856965c36a24e339b6058,cama_mesa_banho,488
13782,14051,422879e10f46682990de24d770e7f83d,ferramentas_jardim,484
29737,30293,389d119b48cf3043d311335e499d9c6b,ferramentas_jardim,392
4500,4598,368c6c730842d78016ad823897a372db,ferramentas_jardim,388
...,...,...,...,...
13879,14148,57c2280ea5af7f10ebfc01a5c47b8041,utilidades_domesticas,1
13878,14147,9246f7bd279121747dcb909a41f44df2,cama_mesa_banho,1
13877,14146,077319a417872fe55c94c3883610b874,esporte_lazer,1
13876,14145,82f5ee2c97b22f801282f0242819d957,esporte_lazer,1


### Criamos uma função que filtra o dataframe acima pela categoria, e o id do produto que está sendo visualizado atualmente pelo usuário.

Removemos o item que está sendo visualizado atualmente pelo usuário, e retornamos uma lista com os ids dos 10 produtos mais vendidos

In [110]:
def returnTop10(categoria, idProdutoAtual):
  df10mais = df.loc[(df['product_category_name'] == categoria) & (df['product_id'] != idProdutoAtual)]
  df10mais = df10mais.sort_values(by=['quantity'], ascending=False)
  lista = df10mais['product_id'].head(10)
  return lista.tolist()

Chamada da função, passando a categoria esporte_lazer, que poderia ser feita através de um site por exemplo

In [106]:
returnTop10('esporte_lazer')

['c6336fa91fbd87c359e44f5dca5a90ed',
 'e44f675b60b3a3a2453ec36421e06f0f',
 '362b773250263786dd58670d2df42c3b',
 'd3c044bd42d84a79e3b0c42662806a48',
 'cec09725da5ed01471d9a505e7389d37',
 '11875b30b49585209e608f40e8082e65',
 '44a34214a57dc373dcd80f54c919d006',
 '19421075ae0b585f2dc13ff149e2119d',
 '781afe929e3016a667f5f439afd55fce',
 '054515fd15bc1a2029f10de97ffa9120']

### Como não temos o valor unitário de cada produto, criamos uma função que retorna um dataframe com o valor médio das compras da lista de ids de produtos passadas.

In [130]:
def listaPrecoMedioProdutos(listaIds):
  filtered_df = order_items_products[order_items_products['product_id'].isin(listaIds)]
  filtered_df = filtered_df[['product_id','price']]
  pd.options.display.float_format = 'R${:,.2f}'.format
  media = pd.DataFrame(filtered_df.groupby(['product_id']).mean())
  return media

### Chamamos a função returnTop10 que retorna a lista, e passamos como parametro para a listaPrecoMedioProdutos para exibir os produtos e seus respectivos preços médios

In [131]:
listaPrecoMedioProdutos( returnTop10('esporte_lazer','c6336fa91fbd87c359e44f5dca5a90ed') )

,price
product_id,
054515fd15bc1a2029f10de97ffa9120,R$42.43
11875b30b49585209e608f40e8082e65,R$65.60
19421075ae0b585f2dc13ff149e2119d,R$47.26
362b773250263786dd58670d2df42c3b,R$51.82
44a34214a57dc373dcd80f54c919d006,R$29.50
72172e982e8b92155069e4201c92c0bb,R$32.35
781afe929e3016a667f5f439afd55fce,R$108.76
cec09725da5ed01471d9a505e7389d37,R$69.48
d3c044bd42d84a79e3b0c42662806a48,R$28.04


### Vamos retornar os itens melhor avaliados de uma determinada categoria
Neste caso, como não temos a avaliação realizada por item, vamos filtrar apenas vendas onde foram vendidos apenas 1 produto, onde podemos deduzir que a avaliação da venda se refere aquele produto.

Fazemos a contagem da tabela de vendas, agrupando pelo id da venda

In [134]:
dfCount = pd.DataFrame(order_items['order_id'].value_counts().reset_index())

Filtramos o dataframe pela quantidade de vendas = 1

In [136]:
dfCount = dfCount.loc[dfCount['order_id']==1]

Renomeamos as colunas para não se perder na hora do merge

In [141]:
dfCount.rename(columns={'index':'order_id', 'order_id':'quantity'}, inplace=True)

,order_id,quantity
9803,a8ddfd6f6316a1c31da18f179981ecc6,1
9804,adfc3673a2cbc4c987d02dfdbc8dbe53,1
9805,adfbf6c2a620196f9a3ec747491c72b7,1
9806,a856b544fe56eea444788271f63512d8,1
9807,a8f8f03d4c94b67001e0f1e242ec1679,1


Fazemos o merge da tabela de reviews com a tabela de orders que só tem um item

In [142]:
dfReviews = dfCount.merge(order_reviews, how="left", on="order_id")

In [144]:
pd.reset_option('^display.', silent=True)

Fazemos um novo merge com a tabela de itens vendidos para conseguir o id do item para juntá-lo posteriormente com a tabela products

In [146]:
dfReviews = dfReviews.merge(order_items, how="inner", on="order_id")

Filtramos apenas as colunas desejadas

In [148]:
dfReviews = dfReviews[['product_id','review_score']]

Calculamos a média das avaliações de cada produto

In [150]:
mediaNotaProduto = pd.DataFrame(dfReviews.groupby(['product_id']).mean())

Realizamos um novo merge agora das médias de cada produto com a tabela de produtos para obter as categorias de cada produto

In [154]:
mediaNotaProduto = mediaNotaProduto.merge(product_clean, how="inner", on="product_id")

Filtramos apenas as colunas desejadas

In [157]:
mediaNotaProduto = mediaNotaProduto[['product_id', 'review_score', 'product_category_name_x']]

Criamos uma função que retorna os 10 itens melhor avaliados baseado na categoria passada

Removemos o id do produto observado para que ele não seja recomendado pois já está sendo visualizado

In [164]:
def returnTopReviews(categoria, idProdutoAtual):
  df10mais = mediaNotaProduto.loc[(mediaNotaProduto['product_category_name_x'] == categoria) & (df['product_id'] != idProdutoAtual)]
  df10mais = df10mais.sort_values(by=['review_score'], ascending=False)
  lista = df10mais[['product_id','review_score']].head(10)
  return lista

Chamamos a função enviando a categoria e o ID do produto atual

In [165]:
returnTopReviews('automotivo','00088930e925c41fd95ebfe695fd2655')

,product_id,review_score
14783,81f869ddb8b8e77514002be908adbacc,5.0
15593,89ad4d91b311611f135ef67426267ffc,5.0
15745,8b2a1e0d2ee4fa00cd00519b7881203a,5.0
15698,8abdb2ffbf92eb0bcdd82fce515fde6a,5.0
15660,8a6bebed424788f0e8e407bf8789102b,5.0
15611,89e0afa29b7165f7a6a95381418dd471,5.0
15608,89db899bdb35ae40d0901e38f21d2ede,5.0
15607,89d845908f978f6a8d8b8779ef177216,5.0
15458,883e57b1f0fab96202d9e39efbe3b671,5.0
15784,8b75356a402f017bf41f1000d514f942,5.0
